In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [83]:
BASE_URL = 'https://cyberpolice.gov.ir'
CATEGORY = f'{BASE_URL}/media_knowledge/c?cat=43'
CATEGORY = 'https://cyberpolice.gov.ir/media_knowledge/c?cat=24291'

In [54]:
def create_driver(headless = True):
    options = Options()
    options.headless = headless
    return webdriver.Chrome("/usr/bin/chromedriver", options=options)

In [49]:
def get_post_content(href):
    global driver
    driver.execute_script("window.open();")
    driver.switch_to.window(driver.window_handles[1]) 
    driver.get(href)
    data = driver.execute_script("""
    return document.querySelector(".row.bs-1col-stacked.node").innerText;""")
    driver.close()
    return data

In [52]:
def get_posts(url,contents=[],i=1):
    global driver
    print('page_'+str(i))
    base_window = driver.current_window_handle

    driver.get(url)
    posts = driver.execute_script("""
    return Array.from(document.querySelectorAll(".view-content h2>a")).map(
            item=>item.getAttribute('href')
        ) 
            """)
    next_page = driver.execute_script("""return document.querySelector('a[rel="next"]')""")

    posts = list(map(lambda post:f'{BASE_URL}/{post}',posts))
    for post in posts:
        _ = get_post_content(post)
        DT.loc[len(DT.index)] = _
        contents.append(_)
        driver.switch_to.window(base_window)
      
    if next_page:
        i+=1
        url = next_page.get_attribute("href")
        get_posts(url,contents,i)    
    return contents

In [ ]:
DT = pd.DataFrame(columns=['body'])
driver = create_driver(False)

driver.get(CATEGORY)

# base_window = driver.current_window_handle

# driver.switch_to.window(base_window)
get_posts(CATEGORY,[])  
DT    
driver.close()

In [ ]:
DT['title'] = DT['body'].str.partition('\n')[0]
DT.to_csv('./contents/cyper_02.csv')